In [1]:
import urllib
import requests
import pandas as pd
import json
import time

import module_api_raw_data as ma
import module_read_file as mr
import module_save_file as ms

In [2]:
# task1 extract
# 擷取全國年度的合併csv資料並轉為df
dir_path = "/workspaces/TIR104_g2/A0_raw_data/tw/tw_movie_2022-2025/"
file_name = "TWMovie2022-2025.csv"
file_path = dir_path + file_name
df_tw_annual = mr.read_file_to_df(file_path)

In [9]:
# task2 transform
# 清理 Name欄位名稱-> Name_s
# 去除()以及全形的（）
# 取修復版前面的電影名稱
df_tw_annual["Name_search"] = df_tw_annual["Name"].str.split("(").str[0]
df_tw_annual["Name_search"] = df_tw_annual["Name_search"].str.split("（").str[0]
df_tw_annual["Name_search"] = df_tw_annual["Name_search"].apply(lambda name: name.split(" ")[0] if "修復" in name else name)

# dir_path = "/workspaces/TIR104_g2/A0_raw_data/tw/tw_mapping_tmdb"
# file_name = "df_tw_annual_test.csv"
# ms.save_as_csv(df_tw_annual, file_name, dir_path)

# df_tw_annual = df_tw_annual.loc[:, ["Year", "MovieId", "Name"]]

In [ ]:
# task3 extract
# 針對台灣總表取出 Name 欄位，並搜尋每個內容
# 存入查詢總結果list
query_list = df_tw_annual["Name_search"]
total_search_results = ma.tmdb_list_search_results(query_list)

In [5]:
# task4 tansform
# 針對Name、title新增處理空白、大小寫的欄位
df_tmdb_search_results = pd.DataFrame(total_search_results)
df_tmdb_search_results["title_map"] = df_tmdb_search_results["title"].str.replace(" ", "").str.upper()
df_tw_annual["Name_map"] = df_tw_annual["Name_search"].str.replace(" ", "").str.upper()

In [ ]:
# task5 tansform
# 用處理過的名稱欄位 merge df_tw_annual 跟 df_tmdb_search_results

df_mapping = df_tw_annual.merge(
    # 因為比對結果，會有重複電影名稱，僅保留第一筆
    df_tmdb_search_results.drop_duplicates(subset=["title"]),

    how="left",
    #df_tw_annual
    left_on="Name_map",
    #df_search_results
    right_on="title_map",
)

df_mapping_result = df_mapping[df_mapping["id"].notna()]

In [12]:
# task6 load
# 存原始資料進A0_raw_data
dir_path = "/workspaces/TIR104_g2/A0_raw_data/tw/tw_mapping_tmdb"
file_name = "v2_mapping_close_true.csv"
ms.save_as_csv(df_mapping_result, file_name, dir_path)

v2_mapping_close_true.csv 儲存成功, 存放路徑: /workspaces/TIR104_g2/A0_raw_data/tw/tw_mapping_tmdb/v2_mapping_close_true.csv


In [8]:
nan_count = df_mapping["id"].isna().sum()
success_count = df_mapping["id"].notna().sum()
print(f"比對判定失敗: {nan_count} 筆資料")
print(f"比對判定成功: {success_count} 筆資料")

比對判定失敗: 438 筆資料
比對判定成功: 1830 筆資料
